In [9]:
import pandas as pd
import re
import pandas as pd
import nltk #Import NLTK library
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import string
nltk.download('punkt') #installed punkt to fix error


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gokul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
#reading each text file
with open("C:/Users/Gokul/Documents/IRTM_Project/Books_new/Ponniyin_Selvan_-_Kalki.txt", "r", encoding="utf-8") as file:
    file1 = file.read()
with open("C:/Users/Gokul/Documents/IRTM_Project/Books_new/Whirlwind_part2.txt", "r", encoding="utf-8") as file:
    file2 = file.read()
with open("C:/Users/Gokul/Documents/IRTM_Project/Books_new/ponniyin-selvan-the-killer-sword-part-3.txt", "r", encoding="utf-8") as file:
    file3 = file.read()
with open("C:/Users/Gokul/Documents/IRTM_Project/Books_new/ponniyin-selvan-the-crown-part-4.txt", "r", encoding="utf-8") as file:
    file4 = file.read()
with open("C:/Users/Gokul/Documents/IRTM_Project/Books_new/ponniyin-selvan-the-supreme-sacrifice-part-5.txt", "r", encoding="utf-8") as file:
    file5 = file.read()


### Combined File

In [26]:
file = file1+file2+file3+file4+file5

In [13]:
len(file)

3690376

In [14]:
stopwords = set(stopwords.words('english')) #English stopwords assigned to "stopwords" object

import string #Punctuation
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

stopwords = [''.join(item for item in x if item not in string.punctuation) for x in stopwords]

In [15]:
# Assuming file_content1 is a string containing the text you want to process
stopwords = set(stopwords)  # Convert stopwords to a set for efficient lookup

# Split the string into individual words
words = file.split()

# Remove stopwords from the list of words
filtered_words = [word for word in words if word not in stopwords]

# Join the filtered words back into a string
filtered_file = ' '.join(filtered_words)

# Now you can use filtered_content1, which is the string with stopwords removed


In [16]:
len(filtered_file)

2634869

In [28]:
file = file.replace("\n", " ")

In [59]:
from transformers import AutoTokenizer
from transformers import pipeline

In [35]:


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

C:\Users\Gokul\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Gokul\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [29]:
# Sentence tokenization
sentences = nltk.sent_tokenize(file)

In [30]:
sentences[1]

'We shall cross a century in a convenient second, and reach a year roughly nine hundred and eighty two years before our time.'

In [31]:
# Define the batch size
batch_size = 3

# Formulate the batch_sentences
batch_sentences = []
for i in range(0, len(sentences), batch_size):
    batch = sentences[i : i + batch_size]
    batch_sentences.append(batch)


In [40]:
batch_sentences = [sentence.strip() for sentence in sentences]

In [49]:
encoded_inputs = tokenizer(batch_sentences, padding=True, truncation=True)

In [42]:
print(encoded_inputs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']
token_type_ids = encoded_inputs.get('token_type_ids')

#### Build Tensors

In [51]:
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")


In [53]:
input_ids = encoded_input['input_ids']
attention_mask = encoded_input['attention_mask']
token_type_ids = encoded_input.get('token_type_ids')

In [60]:
# Initialize the sentiment analysis pipeline
nlp = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [61]:
# Initialize counters
positive_count = 0
negative_count = 0

# Process each sentence and update counters
for sentence in sentences:
    result = nlp(sentence)[0]  # Perform sentiment analysis
    label = result['label']  # Get the sentiment label

    if label == "POSITIVE":
        positive_count += 1
    elif label == "NEGATIVE":
        negative_count += 1

# Calculate totals
total_positive_sentences = positive_count
total_negative_sentences = negative_count

# Print the results
print("Total positive sentences:", total_positive_sentences)
print("Total negative sentences:", total_negative_sentences)

Total positive sentences: 24083
Total negative sentences: 28558


In [64]:
# Initialize counters
positive_count = 0
negative_count = 0

# Create lists to store positive and negative sentence results
positive_sentences = []
negative_sentences = []

#Process each sentence and update counters
for sentence in sentences:
    result = nlp(sentence)[0]  # Perform sentiment analysis
    label = result['label']  # Get the sentiment label
    score = round(result['score'], 4)  # Get the sentiment score

    if label == "POSITIVE":
        positive_count += 1
        positive_sentences.append((sentence, score))
    elif label == "NEGATIVE":
        negative_count += 1
        negative_sentences.append((sentence, score))

# Sort positive and negative sentences by score (in descending order)
positive_sentences.sort(key=lambda x: x[1], reverse=True)
negative_sentences.sort(key=lambda x: x[1], reverse=True)

# Print the results
print("Total positive sentences:", positive_count)
print("Total negative sentences:", negative_count)
print("\nTop 5 positive sentences:")
for sentence, score in positive_sentences[:5]:
    print(f"Sentence: {sentence}\nScore: {score}\n")
print("\nTop 5 negative sentences:")
for sentence, score in negative_sentences[:5]:
    print(f"Sentence: {sentence}\nScore: {score}\n")

Total positive sentences: 24083
Total negative sentences: 28558

Top 5 positive sentences:
Sentence: People milled around all of them, enjoying the sights and scenes.
Score: 0.9999

Sentence: This one has an excellent history, and worthy of worship, as well.
Score: 0.9999

Sentence: Welcome, welcome, to Rains, Wealth and Fortune!]
Score: 0.9999

Sentence: The land shall flourish!
Score: 0.9999

Sentence: I'm impressed.
Score: 0.9999


Top 5 negative sentences:
Sentence: You must be very tired.
Score: 0.9998

Sentence: “Really, I fail to the see the point of these petty arguments.
Score: 0.9998

Sentence: This comprehensively rude argument did not seem to have any effect on the sanyasi.
Score: 0.9998

Sentence: But ah—what a disappointment!
Score: 0.9998

Sentence: You needn't entertain any suspicions on that score.
Score: 0.9998



In [9]:
!pip install flair

     -------------------------------------- 373.1/373.1 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 1.6/1.6 MB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 53.1/53.1 kB 2.7 MB/s eta 0:00:00
     -------------------------------------- 788.5/788.5 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------------------------------------- 1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qiskit-machine-learning 0.6.0 requires dill<0.3.6,>=0.3.4, but you have dill 0.3.6 which is incompatible.
